<a href="https://colab.research.google.com/github/developer0hye/ColabXPytorch-Image-Classfier/blob/master/training_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!unzip /content/drive/"My Drive"/dataset/Kuroko_X_Kagami.zip -d /content/drive/"My Drive"/dataset/

Archive:  /content/drive/My Drive/dataset/Kuroko_X_Kagami.zip
replace /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_30.PNG? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_30.PNG  
  inflating: /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_31.PNG  
  inflating: /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_32.PNG  
  inflating: /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_33.PNG  
  inflating: /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_34.PNG  
  inflating: /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_35.PNG  
  inflating: /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_36.PNG  
  inflating: /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_37.PNG  
  inflating: /content/drive/My Drive/dataset/Kuroko_X_Kagami/test/kagami/kagami_38.PNG  
  inflating: /

# Model

In [0]:
from torch import nn

class SuperLightMobileNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(SuperLightMobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )
        self.num_classes = num_classes
        self.model = nn.Sequential(
            conv_bn(  3,  16, 2), 
            conv_dw( 16,  32, 1),
            conv_dw( 32, 64, 2),
            conv_dw(64, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 1)
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, self.num_classes)

    def forward(self, x):
        x = self.model(x)
        x = self.gap(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

# Hyperprameters & Device Setting

In [0]:
import torch

#hyp parameters
dataset_path = "/content/drive/My Drive/dataset/Kuroko_X_Kagami"
model_weight_save_path = "/content/drive/My Drive/model/"
num_classes = 2

batch_size = 16
num_workers = 8
lr = 1e-3

total_epoch = 30

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Dataset & Dataloader Setting

In [0]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
import torchvision.datasets as datasets
import os


# Data loading code
traindir = os.path.join(dataset_path, 'train')
testdir = os.path.join(dataset_path, 'test')

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                   std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=True, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=True)

# Model Initialization

In [0]:
model = SuperLightMobileNet(num_classes).to(device)

# Loss & Optimizer

In [0]:
CEloss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Training & Evaluation

In [28]:
import numpy as np

total_iteration_per_epoch = int(np.ceil(len(train_dataset)/batch_size))

for epoch in range(1, total_epoch + 1):
    model.train()
    for itereation, (input, target) in enumerate(train_loader):
        images = input.to(device)
        labels = target.to(device)

        # Forward pass
        outputs = model(images)
        loss = CEloss(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch [{}/{}], Iteration [{}/{}] Loss: {:.4f}'.format(epoch, total_epoch, itereation+1, total_iteration_per_epoch, loss.item()))
    if epoch % 10 == 0:
      torch.save(model.state_dict(), model_weight_save_path + 'model_' + str(epoch) + ".pth")
    
    model.eval()
    with torch.no_grad():
      correct = 0
      total = 0
      for input, target in test_loader:
          images = input.to(device)
          labels = target.to(device)

          # Forward pass
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += len(labels)
          correct += (predicted == labels).sum().item()

      print('Epoch [{}/{}], Test Accuracy of the model on the {} test images: {} %'.format(epoch, total_epoch, total, 100 * correct / total))

Epoch [35/50], Iteration [1/7] Loss: 0.0011
Epoch [35/50], Iteration [2/7] Loss: 0.0065
Epoch [35/50], Iteration [3/7] Loss: 0.0031
Epoch [35/50], Iteration [4/7] Loss: 0.0003
Epoch [35/50], Iteration [5/7] Loss: 0.1244
Epoch [35/50], Iteration [6/7] Loss: 0.0468
Epoch [35/50], Iteration [7/7] Loss: 0.0215
Epoch [35/50], Test Accuracy of the model on the 20 test images: 100.0 %
Epoch [36/50], Iteration [1/7] Loss: 0.0007
Epoch [36/50], Iteration [2/7] Loss: 0.0052
Epoch [36/50], Iteration [3/7] Loss: 0.0175
Epoch [36/50], Iteration [4/7] Loss: 0.0043
Epoch [36/50], Iteration [5/7] Loss: 0.0000
Epoch [36/50], Iteration [6/7] Loss: 0.0003
Epoch [36/50], Iteration [7/7] Loss: 0.0095
Epoch [36/50], Test Accuracy of the model on the 20 test images: 100.0 %
Epoch [37/50], Iteration [1/7] Loss: 0.0000
Epoch [37/50], Iteration [2/7] Loss: 0.0068
Epoch [37/50], Iteration [3/7] Loss: 0.0001
Epoch [37/50], Iteration [4/7] Loss: 0.0003
Epoch [37/50], Iteration [5/7] Loss: 0.0004
Epoch [37/50], Ite